In [16]:
import csv
import sys
import pandas as pd
import os
import glob
import itertools
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import math

path = r'C:\Users\cchak\Desktop\Data_ECML\VDI Traces\selected_few'
all_files = glob.glob(os.path.join(path, "*"))
print(len(all_files))

def dataprep(f):
    page_size = 4096
    write_size = 1
    counter = -1
    print("Working with file:" + str(f))
    cols = ['Timestamp','Response','IOType','LUN','Offset','Size']
    df = pd.read_csv(f,engine='python',skiprows =1,header=None,na_values=['-1'], index_col=False)
    df.columns = cols
    print("Number of IO Accesses: ",len(df))
    
    lba_list = df['Offset'].tolist()
    size_list = df['Size'].tolist()
    new_lba_list = []
    new_size_list = []

    # Create empty dataframe
    df_new = pd.DataFrame(columns = ['LBA','Size'])

    while(counter < len(lba_list) - 1):
        counter = counter + 1
        size = size_list[counter]
        lba = lba_list[counter]
        num_writes = math.ceil(size/page_size)
        start = 1
        while(start <= num_writes):
            new_lba_list.append(lba)
            new_size_list.append(write_size)
            lba = lba + 1
            start = start + 1

    df_new['LBA'] = new_lba_list
    df_new['Size'] = new_size_list
    file_name_write = f + str('_dataprep_expanded.csv')
    df.to_csv(file_name_write)
    print(file_name_write)

    
    # Compute death time per LBA and add a column
    death_time_list = []
    counter = -1

    while(counter < len(new_lba_list) - 2):
        counter = counter + 1
        if(counter >10000 and counter%10000==0):
            print("Counter reached :"+ str(counter) +"   Total IO: " + str(len(new_lba_list)))

        lba = new_lba_list[counter]

        if(new_lba_list.count(lba) == 1):
            death_time_list.append(-1)
        else:
            next_lba_counter = counter + 1 
            next_lba = new_lba_list[next_lba_counter]
            death_counter = 1
            while(next_lba != lba):
                next_lba_counter = next_lba_counter + 1
                next_lba = new_lba_list[next_lba_counter]
                death_counter = death_counter + 1
                if(next_lba_counter == len(new_lba_list) - 1):
                    break
            if(next_lba_counter >= len(new_lba_list) - 2):
                death_time_list.append(-1)
            else:
                death_time_list.append(death_counter)

                
    df_out = pd.DataFrame(columns = ['LBA','Death_time'])

    while(len(death_time_list) !=len(new_lba_list)):
        new_lba_list.pop()

    df_out['LBA'] = new_lba_list
    df_out['Death_time'] = death_time_list
    df_out.to_csv(f + "_deathtime_added.csv")
    print(f, ' ---  Data Prep Done')

for f in all_files:
    dataprep(f)
    

    

12
Working with file:C:\Users\cchak\Desktop\Data_ECML\VDI Traces\selected_few\2016022719-LUN3.csv
Number of IO Accesses:  782802
C:\Users\cchak\Desktop\Data_ECML\VDI Traces\selected_few\2016022719-LUN3.csv_dataprep_heiner.csv
Counter reached :20000   Total IO: 5268160
Counter reached :30000   Total IO: 5268160
Counter reached :40000   Total IO: 5268160
Counter reached :50000   Total IO: 5268160
Counter reached :60000   Total IO: 5268160
Counter reached :70000   Total IO: 5268160
Counter reached :80000   Total IO: 5268160
Counter reached :90000   Total IO: 5268160
Counter reached :100000   Total IO: 5268160
Counter reached :110000   Total IO: 5268160
Counter reached :120000   Total IO: 5268160
Counter reached :130000   Total IO: 5268160
Counter reached :140000   Total IO: 5268160
Counter reached :150000   Total IO: 5268160
Counter reached :160000   Total IO: 5268160
Counter reached :170000   Total IO: 5268160
Counter reached :180000   Total IO: 5268160
Counter reached :190000   Total IO

KeyboardInterrupt: 